# Kalman filter brute force paramater search

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.formula.api as sm
import statsmodels.tsa.stattools as ts
import statsmodels.tsa.vector_ar.vecm as vm

import time
from IPython.display import clear_output

In [2]:
# df1 = pd.read_csv("../data/ZECUSDT-minute.csv", index_col=0, parse_dates=True)
# df2 = pd.read_csv("../data/XMRUSDT-minute.csv", index_col=0, parse_dates=True)
df1 = pd.read_csv("../data/testing-hourly/CELRUSDT-minute.csv", index_col=0, parse_dates=True)
df2 = pd.read_csv("../data/testing-hourly/FETUSDT-minute.csv", index_col=0, parse_dates=True)
print(len(df1))
print(len(df2))

1212637
1247928


In [18]:
df = df1.close.rename("A").to_frame()
df["B"] = df2.close
df = df[4000:]
df = df.dropna()
results = sm.ols(formula="B ~ A", data=df[['B', 'A']]).fit()
hedgeRatio = results.params[1]

In [19]:
def get_result(l, thres, s_thres, fee=0.003):
    """generates the profit
    l is length/lookback period, used for spread's mean and std"""
    spread = pd.Series((df['B'] - hedgeRatio * df['A'])).rename("spread").to_frame()
    spread["mean"] = spread.spread.rolling(l).mean()
    spread["std"] =  spread.spread.rolling(l).std()
    spread["zscore"] = pd.Series((spread["spread"]-spread["mean"])/spread["std"])
    spread = spread.dropna()
    spread["A"] = df1["close"].reindex(spread.index)
    spread["Ah"] = df1["high"].reindex(spread.index)
    spread["Al"] = df1["low"].reindex(spread.index)

    spread["B"] = df2["close"].reindex(spread.index)
    spread["Bh"] = df2["high"].reindex(spread.index)
    spread["Bl"] = df2["low"].reindex(spread.index)
    return run_backtest(spread, thres, s_thres, fee)
    
def run_backtest(spread, thres, s_thres, fee):
    # Buy  random value between high and low
    total = 100.
    fee = fee       #Set double normal rate
    thres = thres         #2
    sell_thres = s_thres   # 0, in range [0, ...) for more travel
    price_a = None
    price_b = None
    long = None       #Values: None, "A", "B"
    trades = 0
    for i in range(spread.shape[0]):
        z = spread.zscore[i]
        if long == None: # Looking to buy
            if z>thres or z<-thres:
                trades += 1
                al, ah, bl, bh = get_a_b(spread.Al[i], spread.A[i], spread.Ah[i], spread.Bl[i], spread.B[i], spread.Bh[i])
                if z>thres: 
                    price_a = [ah, ((total/2)/ah)*(1.-fee)]
                    price_b = [bl, ((total/2)/bl)*(1.-fee)]
                    long = "B"
                elif z<-thres:
                    price_a = [al, ((total/2)/al)*(1.-fee)]
                    price_b = [bh, ((total/2)/bh)*(1.-fee)]
                    long = "A"
        if long == "A" and z>sell_thres: #Liquidate positions
            al, ah, bl, bh = get_a_b(spread.Al[i], spread.A[i], spread.Ah[i], spread.Bl[i], spread.B[i], spread.Bh[i])
            total += (price_a[0] - ah)*price_a[1]-(ah*price_a[1]*fee)
            total += (bl - price_b[0])*price_b[1]-(bl*price_b[1]*fee)
            price_a, price_b, long = None, None, None
        elif long == "B" and z<-sell_thres: #Liquidate positions
            al, ah, bl, bh = get_a_b(spread.Al[i], spread.A[i], spread.Ah[i], spread.Bl[i], spread.B[i], spread.Bh[i])
            total += (al - price_a[0])*price_a[1]-(al*price_a[1]*fee)
            total += (price_b[0] - bh)*price_b[1]-(bh*price_b[1]*fee)
            price_a, price_b, long = None, None, None
    return total, trades
    
def get_a_b(al, ac, ah, bl, bc, bh):
    return ac-abs(ac-al)/2, ac+abs(ac-ah)/2, bc-abs(bc-bl)/2, bc+abs(bc-bh)/2

In [5]:
start = time.time()
get_result(6000, 2, -1)
time.time()-start

13.310431003570557

In [12]:
d = {"l":[], "thres":[], "sell_thres":[], "trades":[], "result":[]}
for l in [500, 1000, 2000, 4000, 6000, 8000, 10000, 13000, 15000, 20000, 30000, 50000]:
    for thres in [0.4, 0.7, 1, 1.3, 1.7, 2., 2.3, 2.6, 2.9, 3.2]:
        for sell_thres in [-1.8, -1.5, -1.2, -0.9, -0.6, -0.3, 0., 0.3, 0.6, 0.9, 1.2, 1.5, 2.]:
            if sell_thres <= -thres:
                continue
            print(f"Now doing l: {l}, thres: {thres}, sell_thres: {sell_thres}, safe to kill kernel")
            result, trades = get_result(l, thres, sell_thres)
            print(f"DON'T KILL KERNEL, ADDING TRADES")
            time.sleep(1)
            d["trades"].append(trades)
            d["result"].append(result)
            d["l"].append(l)
            d["thres"].append(thres)
            d["sell_thres"].append(sell_thres)
            clear_output()
            print(pd.DataFrame(d).sort_values("result", ascending=False).iloc[0])
                

l             2.000000e+03
thres         4.000000e-01
sell_thres    6.000000e-01
trades        2.548000e+03
result        2.561568e+06
Name: 233, dtype: float64
Now doing l: 6000, thres: 1.3, sell_thres: 0.0, safe to kill kernel


KeyboardInterrupt: 

In [14]:
df = pd.DataFrame(d)

In [15]:
df.sort_values("result", ascending=False).head(20)

,l,thres,sell_thres,trades,result
233,2000,0.4,0.6,2548,2.561568e+06
242,2000,0.7,0.6,2243,1.513237e+06
128,1000,0.7,0.9,2850,8.085268e+05
243,2000,0.7,0.9,1504,7.602887e+05
151,1000,1.3,1.5,1593,6.977237e+05
138,1000,1.0,0.9,2634,6.512699e+05
252,2000,1.0,0.6,1773,6.365216e+05
234,2000,0.4,0.9,1521,6.113817e+05
119,1000,0.4,0.9,2904,6.052345e+05
140,1000,1.0,1.5,1610,5.667849e+05


In [ ]:
d = {"l":[], "thres":[], "sell_thres":[], "trades":[], "result":[]}
for l in [1400, 1600, 1800, 2000, 2200, 2400, 2600]:
    for thres in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.]:
        for sell_thres in [0.5, 0.6, 0.7, 0.8, 0.9, 1.]:
            if sell_thres <= -thres:
                continue
            print(f"Now doing l: {l}, thres: {thres}, sell_thres: {sell_thres}, safe to kill kernel")
            result, trades = get_result(l, thres, sell_thres)
            print(f"DON'T KILL KERNEL, ADDING TRADES")
            time.sleep(1)
            d["trades"].append(trades)
            d["result"].append(result)
            d["l"].append(l)
            d["thres"].append(thres)
            d["sell_thres"].append(sell_thres)
            clear_output()
            print(pd.DataFrame(d).sort_values("result", ascending=False).iloc[0])
                

l             1.800000e+03
thres         6.000000e-01
sell_thres    8.000000e-01
trades        1.950000e+03
result        3.179701e+06
Name: 135, dtype: float64
Now doing l: 2400, thres: 0.5, sell_thres: 0.7, safe to kill kernel
